Referências - https://www.macroption.com/black-scholes-formula/

In [7]:
from tracemalloc import start
import pandas as pd
import numpy as np
import scipy
import yfinance as yf
from scipy.stats import norm
import trading_calendars as tc

N = norm.cdf
N_ = norm.pdf

from numpy import log as ln
import datetime as dt

ticker = 'PETR4.SA'
start_date = '2021-08-24'
end_date = '2022-08-26'
df = yf.Ticker(ticker).history(start=start_date, end=end_date, auto_adjust=False)

S = df['Close']
S0 = S[-1] #preço de fechamento do dia 2022-08-25
q = 0 #sem dividend yield
r = 0.1375 #Selic
log_returns = ln(S).diff()[1:]
sigma = (log_returns.std())*np.sqrt(252)

In [8]:
options = pd.read_excel('petr4_options.xlsx')[['Ticker', 'Tipo', 'Strike', 'Último']]
options

,Ticker,Tipo,Strike,Último
0,PETRI375,CALL,32.02,2.37
1,PETRU375,PUT,32.02,0.82
2,PETRU387,PUT,32.27,0.92
3,PETRI390,CALL,32.52,1.97
4,PETRU390,PUT,32.52,0.85
5,PETRU385,PUT,32.77,1.10
6,PETRI297,CALL,33.02,1.76
7,PETRU297,PUT,33.02,1.23
8,PETRU392,PUT,33.27,1.34
9,PETRI167,CALL,33.52,1.46


In [9]:
total_de_sessoes_ibov = len(tc.get_calendar("BVMF").sessions_in_range('2022-08-25', '2022-09-16'))
t = (1/252)*total_de_sessoes_ibov
K = 32.02

In [10]:
class black_scholes:
    """
        Modelo de Black-Scholes, calcular o preço de uma call/put option. Premissas do modelo:
            - A taxa de juros é constante e conhecida;
            - O preço do ativo segue um movimento browniano geométrico;
            - Não há custos de transação;

        Parâmetros:
            S0 - Preço/unidade inicial do ativo S
            sigma - Volatilidade do retorno logaritmico do ativo S
            K - Strike price do ativo S
            d - Dividend Yield
            r - Taxa livre de risco
            T - Tempo para maturar a opção (em anos)
    """
    def __init__(self, S0, sigma, K, q, r, t) -> None:
        self.S0 = S0
        self.sigma = sigma
        self.K = K
        self.q = q
        self.r = r
        self.t = t

        d_1 = ( ln(S0/K) + t*(r - q + ((sigma**2)/2) ) )/(sigma*np.sqrt(t))
        d_2 = d_1 - sigma*np.sqrt(t)

        self.d_1 = d_1
        self.d_2 = d_2
        pass

    def get_call_option_price(self):
        C = self.S0*np.exp(-self.q*self.t)*N(self.d_1) - self.K*np.exp(-self.r*self.t)*N(self.d_2)
        return C
    
    def get_call_delta(self):
        delta_c = np.exp(-self.q*self.t)*N(self.d_1)
        return delta_c
    
    def get_call_theta(self):
        theta_c = (1/self.t)*( - ((self.S0*self.sigma*np.exp(-self.q*self.t))/2*np.sqrt(self.t))*N_(self.d_1) \
            - self.r*self.K*np.exp(-self.r*self.t)*N(self.d_2) + self.q*self.S0*np.exp(-self.q*self.t)*N(self.d_1))
        return theta_c
    
    def get_call_rho(self):
        call_c = (1/100)*self.K*self.t*np.exp(-self.r*self.t)*N(self.d_2)
        return call_c
    
    def get_put_option_price(self):
        P = self.K*np.exp(-self.r*self.t)*N(-self.d_2) - self.S0*np.exp(-self.q*self.t)*N(-self.d_1)
        return P

    def get_put_delta(self):
        delta_p = np.exp(-self.q*self.t)*(N(self.d_1) - 1)
        return delta_p
    
    def get_put_theta(self):
        theta_p = (1/self.t)*( - ((self.S0*self.sigma*np.exp(-self.q*self.t))/2*np.sqrt(self.t))*N_(self.d_1) \
            + self.r*self.K*np.exp(-self.r*self.t)*N(-self.d_2) - self.q*self.S0*np.exp(-self.q*self.t)*N(-self.d_1))
        return theta_p
    
    def get_put_rho(self):
        call_p = -(1/100)*self.K*self.t*np.exp(-self.r*self.t)*N(-self.d_2)
        return call_p
    
    def get_gamma(self):
        gamma = (np.exp(-self.q*self.t)/(self.S0*self.sigma*np.sqrt(self.t)))*N_(self.d_1)
        return gamma
    
    def get_vega(self):
        vega = (1/100)*self.S0*np.exp(-self.q*self.t)*np.sqrt(self.t)*N_(self.d_1)
    
    

In [11]:
q = 0
print(f"S0 (último preço do ativo): {S0}")
print(f"sigma (Volat anual do retorno logaritmico): {sigma}")
print(f"K (Preço de Strike): {K}")
print(f"q (Dividendd yield a.a.): {q}")
print(f"r (Selic a.a.): {r}")
print(f"t (tempo em anos até maturação): {t}")

S0 (último preço do ativo): 33.279998779296875
sigma (Volat anual do retorno logaritmico): 0.42107377981983835
K (Preço de Strike): 32.02
q (Dividendd yield a.a.): 0
r (Selic a.a.): 0.1375
t (tempo em anos até maturação): 0.06349206349206349


In [12]:
print(f"Preço da call: {black_scholes(S0, sigma, K, q, r, t).get_call_option_price():.4f}")

print(f"Preço do put: {black_scholes(S0, sigma, K, q, r, t).get_put_option_price():.4f}")

Preço da call: 2.2791
Preço do put: 0.7408
